# [A continuous identity verification method based on free-text keystroke dynamics](http://ieeexplore.ieee.org.proxy1.lib.uwo.ca/document/7844242/authors)

In [1]:
import psycopg2
import pandas as pd
import sys
sys.path.append("..")
import credentials
from sqlalchemy import create_engine
engine = create_engine(credentials.postgres)
import numpy as np
from datetime import datetime
import time
import string
from tqdm import tnrange, tqdm_notebook, tqdm
import numpy.ma as ma
from multiprocessing import Pool
# This makes plots render inline

In [2]:
# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm_notebook().pandas(desc="Progress")

In [3]:
% matplotlib inline

In [4]:
key_codes = pd.read_csv('key_codes.csv').set_index('code').key

In [5]:
# Basic Connection and Query
connection = engine.connect()

In [6]:
def get_sequence(user_id=None, limit=None, connection=connection, filter_alpha=False):
    people = {}
    # Calculate Dwells
    condition_text = 'WHERE user_id=\'%s\'' % user_id if user_id is not None else ''
    limit_text = 'LIMIT \'%s\'' % str(limit) if limit is not None else ''
    print("SELECT * FROM strokes %s ORDER BY key_time %s" % (condition_text, limit_text))
    result = connection.execute("SELECT * FROM strokes %s ORDER BY key_time %s" % (condition_text, limit_text)).fetchall()
    dwells = []
    for i in tnrange(len(result), desc='Loading'):
        row = result[i]
        row = (row[0], row[1], int(row[2]), int(row[3]), row[4])
        # Assure Users are in Dictionaries
        if row[0] not in people:
            people[row[0]] = {}
            
        if row[2] not in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'u':
            dwell = row[1] - people[row[0]][row[2]]
            try:
                if not filter_alpha or (filter_alpha and key_codes[row[2]] in string.ascii_uppercase):
                    dwells.append({
                        'person': row[0],
                        'key_code': row[2],
                        'key': key_codes[row[2]],
                        'down': people[row[0]][row[2]],
                        'up': row[1],
                        'dwell': dwell
                    })
            except Exception as e:
                pass
                
            del(people[row[0]][row[2]])
    return pd.DataFrame(dwells).sort_values('down')


In [7]:
df = get_sequence(user_id='1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f', filter_alpha=True)

SELECT * FROM strokes WHERE user_id='1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f' ORDER BY key_time 


In [22]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


In [23]:
S = df.key
Interval = (df.shift(-1).down - df.up)
orderedInterval = Interval.sort_values().reset_index(drop=True)

In [49]:
required = pd.DataFrame({i: Interval.shift(-i) for i in range(10)}).cummax(axis=1).dropna()

In [81]:
keys = pd.DataFrame({
    i: df.key.shift(-i) for i in range(10)
}).fillna('').progress_apply(lambda x: ['-'.join(x[:i+2]) for i in x.index], axis=1)

In [103]:
seqs = pd.DataFrame({
    i: df.key_code.shift(-i) for i in range(10)
}).apply(lambda x: x*(100**x.name), axis=0).cumsum(axis=1).dropna()

In [108]:
a = tf.Variable([200], dtype=tf.float32)
k = tf.placeholder(tf.int32)
Sa = tf.placeholder(tf.float32)

M = tf.reduce_sum(tf.cast(tf.less(Sa, a), tf.int32), axis=1) - 1
start = tf.range(tf.size(M))
end = start + M

sli = tf.slice(k, 0, M, axis=1)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
sess.run(sli, {Sa: required, k: seqs})

TypeError: slice() got an unexpected keyword argument 'axis'